<a href="https://colab.research.google.com/github/Marieta3/sentiment-analysis-russian/blob/master/sentiment_analiza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Analiza sentimenta tvitova na ruskom jeziku


##Importovanje potrebnih biblioteka

In [2]:
import tensorflow as tf
import tflearn
import pandas as pd
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import RussianStemmer
from nltk.tokenize import TweetTokenizer
import re
from collections import Counter
from datetime import datetime
import numpy as np
from tflearn.data_utils import to_categorical

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


##Konstante

In [0]:
VOCAB_SIZE = 5000
MANJI_DEO = True
TEST_SIZE = 0.3

##Učitavanje podataka

In [0]:
col = 3 #kolona koja sadrzi tvit, ostale kolone nisu potrebne

if(MANJI_DEO):
  positive = pd.read_csv(
      'https://raw.githubusercontent.com/Marieta3/sentiment-analysis-russian/master/positive.csv', header=None, delimiter=';')[[col]].sample(40000);
  negative = pd.read_csv(
      'https://raw.githubusercontent.com/Marieta3/sentiment-analysis-russian/master/negative.csv', header=None, delimiter=';')[[col]].sample(40000);
else:
  positive = pd.read_csv(
      'https://raw.githubusercontent.com/Marieta3/sentiment-analysis-russian/master/positive.csv', header=None, delimiter=';')[[col]];
  negative = pd.read_csv(
      'https://raw.githubusercontent.com/Marieta3/sentiment-analysis-russian/master/negative.csv', header=None, delimiter=';')[[col]];  

##Testiranje učitanih podataka


In [5]:
print('Pozitivnih tvitova: ' +str(len(positive)))
print('Negativnih tvitova: ' +str(len(negative)))

#positive = positive.head(30000)
#negative = negative.head(30000)

print('Pozitivnih tvitova: ' +str(len(positive)))
print('Negativnih tvitova: ' +str(len(negative)))

Pozitivnih tvitova: 40000
Negativnih tvitova: 40000
Pozitivnih tvitova: 40000
Negativnih tvitova: 40000


##Izvlačenje osnovnog oblika iz tokena

In [6]:
stemmer = RussianStemmer()
stem_cache={}  #smanjuje vreme izvršavanja, ako se reč već pojavljivala, samo je uzmemo iz liste
def get_stem(token):
  stem =stem_cache.get(token, None)
  if stem:
    return stem
  
  token=re.sub('[^а-яА-Я ]', '', token)  #samo slova
  stem=stemmer.stem(token)
  stem_cache[token]=stem
  return stem
get_stem('!')

''

##Tokenizacija

In [0]:
recnik = Counter()
tokenizer = TweetTokenizer()
stop_words = stopwords.words('russian')

def popuni_recnik(tweets):
  for t in tweets.values.tolist():
    #print(t[0])
    tweet=t[0]
    
    tokens=tokenizer.tokenize(tweet)
    for token in tokens:
      stem=get_stem(token)
      if stem!='' and stem not in stop_words and token not in stop_words:
        recnik[stem]+=1
    
start_time=datetime.now()  
popuni_recnik(positive)
popuni_recnik(negative)
end_time=datetime.now()

In [9]:
print('Потребно време за попуњавање речника: ', end_time-start_time)
print("Укупан број различитих речи: ", len(recnik))


Потребно време за попуњавање речника:  0:00:39.033091
Укупан број различитих речи:  50340


In [0]:
vocab = sorted(recnik, key=recnik.get, reverse=True)[:VOCAB_SIZE]  #сортирано по броју понављања опадајуће, првих 5000

In [11]:
print(vocab[:10])
print(vocab[1000:1020])
print(vocab[-10:])
print("Дужина скраћеног речника: ",len(vocab))

['эт', 'сегодн', 'прост', 'мо', 'хоч', 'очен', 'ден', 'теб', 'год', 'нов']
['клуб', 'дух', 'карт', 'ю', 'добав', 'завид', 'дает', 'поезд', 'везет', 'золот', 'зовут', 'брос', 'вкус', 'мнен', 'возвраща', 'жил', 'стен', 'пипец', 'однак', 'хз']
['маразм', 'блях', 'отб', 'вока', 'штол', 'стрим', 'форсаж', 'скушн', 'крыс', 'безразличн']
Дужина скраћеног речника:  5000


In [13]:
idx=5000-1
print("Reč: ", vocab[idx], " se pojavila ", recnik.get(vocab[idx]), " puta")




Reč:  безразличн  se pojavila  12  puta


In [14]:
token_to_idx = {vocab[i] : i for i in range(VOCAB_SIZE)}
print(token_to_idx.get('', None))
print(type(token_to_idx))

None
<class 'dict'>


##Pretvaranje tvita u vektor

*   vektor je duzine broja reci u recniku (5000)
*   ukoliko tvit sadrzi rec koja se nalazi u recniku, na odgovarajucem mestu u vektoru se nalazi jedinica; u suprotnom nula


In [0]:
def tweet_to_vector(tweet, print_unknown=False):
  vector=np.zeros(VOCAB_SIZE, dtype=int)
  for token in tokenizer.tokenize(tweet):
    stem=get_stem(token)
    idx=token_to_idx.get(stem, None)
    if idx is not None:
      vector[idx]=1
    else:
      if(print_unknown):
        print('Nepoznata rec: ', token)
  return vector
    

In [16]:
tw_idx=1
pos_tws=positive.values.tolist()
neg_tws=negative.values.tolist()
print("tweet: {}".format(pos_tws[tw_idx][0]))
print("tweet: {}".format(tweet_to_vector(pos_tws[tw_idx][0])[:10]))
print("tweet: {}".format(neg_tws[tw_idx][0]))
print("tweet: {}".format(tweet_to_vector(neg_tws[tw_idx][0])[:10]))

tweet: Представь такую ситуацию,у тебя есть дома кот как ты его назовешь ?)) — Себастьян :3 http://t.co/emLE8R1xbl
tweet: [0 0 0 0 0 0 0 1 0 0]
tweet: RT @tina_kandelaki: Господи, поколение 20-25 дико инфантильное. Не думала, что буду ворчать, как бабка. Но, by the way(
tweet: [0 0 0 0 0 0 0 0 0 0]


In [17]:
try:
  del train_vectors
  del test_vectors
except:
  print('Nije definisano')
  
train_size = round((len(negative) + len(positive))*(1-TEST_SIZE))
test_size = len(negative) + len(positive) - train_size

train_vectors = np.zeros((train_size, VOCAB_SIZE), dtype=np.int_)
test_vectors = np.zeros((test_size, VOCAB_SIZE), dtype=np.int_)

Nije definisano


In [0]:
tweets=[]
cnt=0
cnt2=0

train_pos_size = round(len(positive)*(1-TEST_SIZE))
train_neg_size = round(len(negative)*(1-TEST_SIZE))

for tweet in negative.values.tolist():
  #print(tweet[0])
  tweets.append(tweet[0])
  if cnt < train_neg_size:
    train_vectors[cnt]=tweet_to_vector(tweet[0])
    cnt+=1
  else:
    test_vectors[cnt2]=tweet_to_vector(tweet[0])
    cnt2+=1
for tweet in positive.values.tolist():
  tweets.append(tweet[0])
  if cnt < train_neg_size + train_pos_size:
    train_vectors[cnt]=tweet_to_vector(tweet[0])
    cnt+=1
  else:
    test_vectors[cnt2]=tweet_to_vector(tweet[0])
    cnt2+=1


In [19]:
print(len(train_vectors))
print(len(train_vectors[0]))
print(train_vectors)

56000
5000
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


##Brisanje vektora tvitova

In [0]:
del train_vectors
del test_vectors

##Labele

In [0]:
#labels = np.append(np.zeros(len(negative), dtype=np.int_), np.ones(len(positive), dtype=np.int_))

train_labels = np.append(
    np.zeros(train_neg_size, dtype=np.int_), 
    np.ones(train_pos_size, dtype=np.int_))

test_labels = np.append(
    np.zeros(len(negative) - train_neg_size, dtype=np.int_), 
    np.ones(len(positive) - train_pos_size, dtype=np.int_))

In [0]:
y_train = to_categorical(train_labels, 2)
y_test = to_categorical(test_labels, 2)


##Kreiranje modela

In [0]:
def build_model(learning_rate=0.1):
    tf.reset_default_graph()
    
    net = tflearn.input_data([None, VOCAB_SIZE])
    net = tflearn.fully_connected(net, 125, activation='ReLU')
    net = tflearn.fully_connected(net, 25, activation='ReLU')
    net = tflearn.fully_connected(net, 2, activation='softmax')
    regression = tflearn.regression(
        net, 
        optimizer='sgd', 
        learning_rate=learning_rate, 
        loss='categorical_crossentropy')
    
    model = tflearn.DNN(net)
    return model

In [23]:

model = build_model(learning_rate=0.75)

W0703 09:28:03.335266 140634864326528 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/layers/core.py:81: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0703 09:28:03.342907 140634864326528 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tflearn/initializations.py:174: calling TruncatedNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0703 09:28:03.406377 140634864326528 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/optimizers.py:135: The name tf.train.GradientDescentOptimizer is deprecated. Please use tf.compat.v1.train.GradientDescentOptimizer instead.

W0703 09:28:03.421644 140634864326528 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tflearn/objectives.py:66: c

In [61]:
model.fit(
    train_vectors, 
    y_train, 
    validation_set=0.1, 
    show_metric=True, 
    batch_size=128, 
    n_epoch=4)

Training Step: 11819  | total loss: 0.03651 | time: 8.328s
| SGD | epoch: 030 | loss: 0.03651 - acc: 0.9839 -- iter: 50304/50400
Training Step: 11820  | total loss: 0.03669 | time: 9.396s
| SGD | epoch: 030 | loss: 0.03669 - acc: 0.9847 | val_loss: 0.05866 - val_acc: 0.9780 -- iter: 50400/50400
--


##Testiranje

In [62]:
predictions = (np.array(model.predict(test_vectors))[:,0] >= 0.5).astype(np.int_)
accuracy = np.mean(predictions == y_test[:,0], axis=0)
print("Accuracy: ", accuracy)

Accuracy:  0.67875


In [0]:
def test_tweet(tweet):
    tweet_vector = tweet_to_vector(tweet, False)
    positive_prob = model.predict([tweet_vector])[0][1]
    print('Original tweet: {}'.format(tweet))
    print('P(positive) = {:.5f}. Result: '.format(positive_prob), 
          'Positive' if positive_prob > 0.5 else 'Negative')

In [0]:
def test_tweet_number(idx):
    test_tweet(tweets[idx])

In [63]:
test_tweet_number(40003)

Original tweet: #Ёлки3 А во-вторых,ну чего Самары-то так мало?)вот этим прямо вообще не довольна.тут ооочень много снимали,получается все труды на смарку!
P(positive) = 0.99997. Result:  Positive


##Proba

In [64]:
tweets_for_testing = [
    "меня оштрафовали по дороге домой!",
    "Я люблю мороженое!",
    "Я не люблю мороженое",
    "меня обманули",
    "меня обманули позитив",
    "меня он очень бесить",
    "меня он не бесить",
    "Почему по ночам мне не спится? У машины скрипит колесо. Отразилась луна в ягодице. 10 лет тебе,синяя птица! Жаль,что букв всего лишь сто со",
    "Смотрите сегодня, в 23:35, сразу после финала #голос6 @voice1tv, на @1tv, большой новогодний выпуск @vecherniy_urgant #вечернийургант! Я, @urgantcom и @lazarevsergey приготовили для вас праздничный новогодний номер!))) #иванургант #сергейлазарев #билан #… http://ift.tt/2zLgzWk ",
    "ШОК! Данилу Козловского задержали в центре Москвы за чтение Шекспира!",
    "Через час в Петербурге,на улице Рубинштейна,дом 23 откроют памятник Сергею Довлатову.Надеваем халаты,тапочки,берем собак и цветы!",
    "Спасибо академикам,коллегам и номинантам!Спасибо всей команде ВУ!Спасибо моей семье!И конечно спасибо Вам,дорогие телезрители!#тэфи2016",
    "А вот и ласточка-новостнуха!24 апреля в Крокусе я поделюсь с Вами инсайтом на Synergy Insight Forum. Приходите что ли! #sif2017"
]
for tweet in tweets_for_testing:
    test_tweet(tweet) 
    print("---------")

Original tweet: меня оштрафовали по дороге домой!
P(positive) = 0.97827. Result:  Positive
---------
Original tweet: Я люблю мороженое!
P(positive) = 0.23612. Result:  Negative
---------
Original tweet: Я не люблю мороженое
P(positive) = 0.23612. Result:  Negative
---------
Original tweet: меня обманули
P(positive) = 0.16907. Result:  Negative
---------
Original tweet: меня обманули позитив
P(positive) = 0.74244. Result:  Positive
---------
Original tweet: меня он очень бесить
P(positive) = 0.00003. Result:  Negative
---------
Original tweet: меня он не бесить
P(positive) = 0.00032. Result:  Negative
---------
Original tweet: Почему по ночам мне не спится? У машины скрипит колесо. Отразилась луна в ягодице. 10 лет тебе,синяя птица! Жаль,что букв всего лишь сто со
P(positive) = 0.00014. Result:  Negative
---------
Original tweet: Смотрите сегодня, в 23:35, сразу после финала #голос6 @voice1tv, на @1tv, большой новогодний выпуск @vecherniy_urgant #вечернийургант! Я, @urgantcom и @lazarev